# Result Cleaning and Evaluation for Binary Cognitive Distortion Detection

In [ ]:
import os
#Make this your results directory
os.chdir('../')
eval_path = '/Users/ulugsali/Desktop/Cognitive-Distortion-Project/results/binary/expert/llama3.2-3b/four-shot.csv'


Open the Dataset

In [124]:
import pandas as pd
inference_df = pd.read_csv(eval_path)
inference_df
results_table = pd.DataFrame()

Fix Common Variations

In [125]:

inference_df['Response'] = inference_df['Response'].apply(lambda x: x.lower())
inference_df['Response'] = inference_df['Response'].apply(lambda x: x.strip(".'"))
inference_df

,Unnamed: 0,Prompt,Response
0,0,My husband works a lot which really helps our ...,yes
1,1,I used to get many strange looks for the thing...,yes
2,2,Moved to another state left everything for my ...,yes
3,3,"It has been more than a year now , I feel alon...",yes
4,4,"My sister has autism spectrum disorder, she al...",yes
...,...,...,...
501,501,From India: My brother is 40 years old and he ...,yes
502,502,From the U.S.: I was sexually abused and raped...,yes
503,503,My grandsons personality has changed in every ...,yes
504,504,From Egypt: I was diagnosed with OCD by my doc...,yes


Create Mapping Function

In [126]:
import re

distortions = {
    'y': 1,
    'n': 0,
    'yes': 1,
    'no': 0
}

patterns = {
    'y': r'\by\b',
    'n': r'\bn\b',
    'yes': r'\byes\b',
    'no': r'\bno\b', 
}

def catch_responses(x):
    normalized_text = re.sub(r'[^\w\s]', '', x.lower())
    for label, pattern in patterns.items():
        if re.search(pattern, normalized_text):
            return distortions[label]
    return -1


Map common desired input, display any that are undesired for fine handling... For me, any denial should be thrown away (by being labelled -1).

In [127]:
inference_df['Response'] = inference_df['Response'].apply(catch_responses)
inference_df.head(50)

,Unnamed: 0,Prompt,Response
0,0,My husband works a lot which really helps our ...,1
1,1,I used to get many strange looks for the thing...,1
2,2,Moved to another state left everything for my ...,1
3,3,"It has been more than a year now , I feel alon...",1
4,4,"My sister has autism spectrum disorder, she al...",1
5,5,I have 4 “versions” of myself. The main one (m...,1
6,6,"Through mental health treatment, I’m being tol...",1
7,7,My girlfriend and i have been dating for 5 yea...,1
8,8,"From India: Initially, I identified myself as ...",1
9,9,Hi. I’m living with extreme guilt and regret....,1


Gather Gold Data

In [128]:
gold_data = pd.read_csv('/Users/ulugsali/Desktop/Cognitive-Distortion-Project/datasets/test.csv')['binary_ground_truth']
inference_df['gold'] = gold_data

In [129]:
results_table[eval_path] = inference_df['Response']

Throw away non-responses

In [130]:
inference_df = inference_df[inference_df['Response'] != -1]

Compute Accuracy and F1

In [131]:
from sklearn.metrics import accuracy_score, f1_score

print(f"Evaluated Model: {eval_path.split('/')[1]} Prompt: {eval_path.split('/')[2].removesuffix('.csv')}")
print(f"Accuracy: {accuracy_score(inference_df['gold'], inference_df['Response'])}")
print(f"F1-Score (Macro): {f1_score(inference_df['gold'], inference_df['Response'], average='macro', pos_label=1)}")
print(f"F1-Score (Binary): {f1_score(inference_df['gold'], inference_df['Response'],average='binary', pos_label=1)}")

Evaluated Model: Users Prompt: ulugsali
Accuracy: 0.6779324055666004
F1-Score (Macro): 0.4040284360189573
F1-Score (Binary): 0.8080568720379147


In [132]:
#results_table['gold'] = gold_data
#results_table.to_csv('/assessment/results_table.csv', index=False)